In [1]:
# spark.sparkContext.getConf().get('spark.driver.memory')
# spark.sparkContext.getConf().get('spark.driver.maxResultSize')
# conf.set("spark.driver.maxResultSize", "3g")

%%configure -f 
{"driverMemory": "6000M"}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1587412236629_0001,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


unknown magic command '%configure'
UnknownMagic: unknown magic command '%configure'



In [1]:
import ast
import numpy as np

bow = spark.read.csv("s3://15405finalprojectcsvdata/bow_data/sparse/formatted").cache()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1587412236629_0002,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
toList = bow.rdd.filter(lambda x: x[0][-1] ==']').map(lambda x: ast.literal_eval(x[0]))
print(toList.count())
print(toList.first())

2476966
[(18947, 1)]

In [3]:
from pyspark.mllib.linalg import Vectors

toSparse = toList.map(lambda x: Vectors.sparse(151951, x))
print(toSparse.count())


2476966

In [4]:
toArray = toSparse.map(lambda x: x.toArray())
toArray.count()

2476966

In [57]:
# meanRdd = mostFreq.map(lambda x: np.mean(x))
# print(meanRdd.count())
# print(meanRdd.first())

2476966
1.5259021896696422e-05

In [42]:
# meanSum = meanRdd.reduce(lambda x, y: x+y)
# dataMean = np.float(meanSum/2476966)
# print(dataMean)

0.000610648590894107

In [47]:
# zeroMean = toArray.map(lambda x: x-dataMean)  # subtract the mean
# print(zeroMean.count())
# print(zeroMean.first())

'PipelinedRDD' object has no attribute 'toDf'
Traceback (most recent call last):
AttributeError: 'PipelinedRDD' object has no attribute 'toDf'



In [50]:
# zeroMeanDf = zeroMean.map(lambda x:(Vectors.dense(x), )).toDF(["features"])


In [56]:
# from pyspark.mllib.stat import Statistics
# # import pandas as pd

# # result can be used w/ seaborn's heatmap
# def compute_correlation_matrix(df, method='pearson'):
#     # wrapper around
#     # https://forums.databricks.com/questions/3092/how-to-calculate-correlation-matrix-with-all-colum.html
#     df_rdd = df.rdd.map(lambda row: row[0:])
#     corr_mat = Statistics.corr(df_rdd, method=method)
#     return corr_mat

# corMat = compute_correlation_matrix(zeroMeanDf)

'Argument with more than 65535 cols: 151951'
Traceback (most recent call last):
  File "<stdin>", line 9, in compute_correlation_matrix
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/mllib/stat/_statistics.py", line 155, in corr
    return callMLlibFunc("corr", x.map(_convert_to_vector), method).toArray()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/mllib/common.py", line 130, in callMLlibFunc
    return callJavaFunc(sc, api, *args)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/mllib/common.py", line 123, in callJavaFunc
    return _java2py(sc, func(*args))
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 79, in deco
    raise IllegalArgumentException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.IllegalArgumentException: 'Argument with more than 65535 cols: 151951'



In [5]:
frequencies = toArray.reduce(lambda x,y: x + y)
frequencies.shape

(151951,)

In [6]:
topFreq = np.argsort(frequencies)[-65535:]
topFreq.shape

(65535,)

In [7]:
mostFreq = toArray.map(lambda x: x[topFreq])
print(mostFreq.count())
print(mostFreq.first())

2476966
[0. 0. 0. ... 0. 0. 0.]

In [9]:
# npArray = mostFreq.collect()

An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 9.0 failed 4 times, most recent failure: Lost task 7.3 in stage 9.0 (TID 502, ip-172-31-36-23.ec2.internal, executor 6): ExecutorLostFailure (executor 6 exited caused by one of the running tasks) Reason: Container from a bad node: container_1587412236629_0001_01_000008 on host: ip-172-31-36-23.ec2.internal. Exit status: 52. Diagnostics: Exception from container-launch.
Container id: container_1587412236629_0001_01_000008
Exit code: 52
Stack trace: ExitCodeException exitCode=52: 
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:972)
	at org.apache.hadoop.util.Shell.run(Shell.java:869)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1170)
	at org.apache.hadoop.yarn.server.nodemanager.DefaultContainerExecutor.launchContainer(DefaultContainerExecutor.java:235)
	at org.apache.hadoop.yarn.s

In [8]:
toDense = mostFreq.map(lambda x: Vectors.dense(x))
print(toDense.count())

2476966

In [15]:
# denseDf = toDense.map(lambda x: (x, )).toDF(["features"])

In [38]:
# type(denseDf)
# denseDf.groupBy().avg('features').collect()

'"features" is not a numeric column. Aggregation function can only be applied on a numeric column.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/group.py", line 42, in _api
    jdf = getattr(self._jgd, name)(_to_seq(self.sql_ctx._sc, cols))
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: '"features" is not a numeric column. Aggregation function can only be applied on a numeric column.;'



In [10]:
from pyspark.mllib.linalg.distributed import RowMatrix

mat = RowMatrix(toDense)


In [11]:
pc = mat.computePrincipalComponents(3)

An error occurred while calling o172.computePrincipalComponents.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 13.0 failed 4 times, most recent failure: Lost task 1.3 in stage 13.0 (TID 552, ip-172-31-36-23.ec2.internal, executor 9): ExecutorLostFailure (executor 9 exited caused by one of the running tasks) Reason: Container from a bad node: container_1587412236629_0002_01_000015 on host: ip-172-31-36-23.ec2.internal. Exit status: 137. Diagnostics: Container killed on request. Exit code is 137
Container exited with a non-zero exit code 137
Killed by external signal
.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2041)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2029)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2028)
	at scala.collection.mutable.Resizab

In [ ]:
# import numpy as np

# def estimateCovariance(rdd):
#     """Compute the covariance matrix for a given dataframe.

#     Note:
#         The multi-dimensional covariance array should be calculated using outer products.  Don't
#         forget to normalize the data by first subtracting the mean.

#     Args:
#         df:  A Spark dataframe with a column named 'features', which (column) consists of DenseVectors.

#     Returns:
#         np.ndarray: A multi-dimensional array where the number of rows and columns both equal the
#             length of the arrays in the input dataframe.
#     """
#     dfZeroMean = rdd.map(lambda x: x[0]-dataMean).toDf(["features"])  # subtract the mean
    
#     return dfZeroMean.map(lambda x: np.outer(x,x)).reduce(lambda x,y: np.add(x,y))/2476966


# cov = estimateCovariance(toArray)
# print(cov.count)()

In [36]:
# # from numpy.linalg import eigh

# def pca(df, k=2):
#     """Computes the top `k` principal components, corresponding scores, and all eigenvalues.

#     Note:
#         All eigenvalues should be returned in sorted order (largest to smallest). `eigh` returns
#         each eigenvectors as a column.  This function should also return eigenvectors as columns.

#     Args:
#         df: A Spark dataframe with a 'features' column, which (column) consists of DenseVectors.
#         k (int): The number of principal components to return.

#     Returns:
#         tuple of (np.ndarray, RDD of np.ndarray, np.ndarray): A tuple of (eigenvectors, `RDD` of
#         scores, eigenvalues).  Eigenvectors is a multi-dimensional array where the number of
#         rows equals the length of the arrays in the input `RDD` and the number of columns equals
#         `k`.  The `RDD` of scores has the same number of rows as `data` and consists of arrays
#         of length `k`.  Eigenvalues is an array of length d (the number of features).
#      """
#     cov = estimateCovariance(df)
#     col = cov.shape[1]
#     eigVals, eigVecs = eigh(cov)
#     inds = np.argsort(eigVals)
#     eigVecs = eigVecs.T[inds[-1:-(col+1):-1]]  
#     components = eigVecs[0:k]
#     eigVals = eigVals[inds[-1:-(col+1):-1]]  # sort eigenvals
#     score = df.select(df['features']).map(lambda x: x[0]).map(lambda x: np.dot(x, components.T) )
#     # Return the `k` principal components, `k` scores, and all eigenvalues

#     return components.T, score, eigVals

In [37]:
# comp, score, eigVals = pca(denseDf)

'DataFrame' object has no attribute 'mean'
Traceback (most recent call last):
  File "<stdin>", line 21, in pca
  File "<stdin>", line 17, in estimateCovariance
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1301, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'mean'



In [ ]:
# Project the rows to the linear space spanned by the top k principal components.
projected = mat.multiply(pc)

In [9]:
type(projected)

name 'projected' is not defined
Traceback (most recent call last):
NameError: name 'projected' is not defined



In [ ]:
# resultDf = projected.rows.map(lambda x: (x, )).toDF()

In [ ]:
# resultDf.coalesce(1).write.format('csv').mode('overwrite').option("header", "false")\
# .save("s3://15405finalprojectcsvdata/pca_transformed_data")

In [ ]:
# from pyspark.mllib.feature import PCA as PCAmllib

# model = PCAmllib(100).fit(toDense)
# transformed = model.transform(toDense)

In [ ]:
# sum_rdd = bow_rdd.map(lambda v: v.toArray())
# sum_rdd.count()

In [ ]:
# lengths.first()

In [ ]:
# types.first()

In [ ]:
# bow.first()

In [ ]:
# rmFirstLast.first()

In [ ]:
# toInt.first()

In [ ]:
# from pyspark.mllib.linalg import Vectors
# toVect = toInt.map(lambda x: Vectors.dense(x)).cache()


In [ ]:
# toVect.first()

In [ ]:
# rows = toVect.collect()
# rows = sc.parallelize(rows)

In [ ]:
# from pyspark.mllib.linalg.distributed import RowMatrix

# mat = RowMatrix(toDense)

# pc = mat.computePrincipalComponents(3)

# # Project the rows to the linear space spanned by the top 4 principal components.
# projected = mat.multiply(pc)

In [ ]:
# from sklearn.decomposition import PCA



# pca = PCA(n_components=3)
# pca_result = pca.fit_transform(df[feat_cols].values)
# df['pca-one'] = pca_result[:,0]
# df['pca-two'] = pca_result[:,1] 
# df['pca-three'] = pca_result[:,2]
# print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))
# Explained variation per principal component: [0.09746116 0.07155445 0.06149531]